# Complete Neuron

## Perameters

- `learning_rate`
- `activation`
- `loss`
- `optimizer`

In [2]:
#
# Imports
#

import numpy as np
from collections.abc import Iterable
from numbers import Number

In [3]:
#
# Customer Exceptions
#

class InvalidInitialWeights(Exception): pass

class UnexpectedInputsShape(Exception): pass
class UnexpectedOutputsShape(Exception): pass

class InvalidActivation(Exception): pass
class InvalidOptimizer(Exception): pass
class InvalidLoss(Exception): pass

In [4]:
#
# Sigmoid Activation
#

class Sigmoid():

  def apply(self, x):
    return 1 / (1 + np.exp(-x))
  
  def derivative(self, x):
    sig = self.apply(x)
    return sig * (1 - sig)

In [5]:
#
# Mean Squared Error
#

class MeanSquaredError():
  N = 0

  def calculate(self, y_true:Iterable, y_pred:Iterable):
    self.N = len(y_true)
    return np.mean(np.square(y_true - y_pred))
  
  def derivative(self, y_true:Number, y_pred:Number):
    return -2*(y_true-y_pred)/self.N

In [6]:
#
# Stochastic Gradient Descent
#

class StochasticGradientDescent():
  def __init__(self, learning_rate=None):
    self.learning_rate = learning_rate

  def update(self, model, learning_rate=0.01):
    # learning rate
    lr = self.learning_rate or model.learning_rate or learning_rate

    # hierarchy functions
    def update_neuron(neuron, lr): neuron.update(lr)
    def update_layer(layer, lr):
      for neuron in layer.neurons: update_neuron(neuron, lr)

    # if ann
    if hasattr(model, 'layers'):
      for layer in model.layers: update_layer(layer, lr)
    
    # if layer
    elif hasattr(model, 'neurons'):
      for neuron in model.neurons: update_neuron(neuron, lr)
    
    # if neuron
    elif hasattr(model, 'forward'):
      update_neuron(model, lr)



In [30]:
class Neuron():

  # 
  # Props
  # 

  builtin:dict = {
    'activation': {
      'default': Sigmoid,
      'sigmoid': Sigmoid,
    },
    'loss_function': {
      'default': MeanSquaredError,
      'sigmoid': MeanSquaredError,
    },
    'optimizer': {
      'default': StochasticGradientDescent,
      'sgd': StochasticGradientDescent,
    }
  }

  errors:dict = {
    'activation':    InvalidActivation,
    'loss_function': InvalidLoss,
    'optimizer':     InvalidOptimizer,
  }

  input:Iterable = []
  output:float = 0.0

  delta:float = 0.0


  weight_gradient:Iterable = []
  bias_gradient:float = 0.0



  # 
  # Constructor
  # 

  def __init__(
      self,

      input_size:int,
      
      learning_rate:float=0.1,
      activation:str|object='default',
      loss_function:str|object='default',
      optimizer:str|object='default',

      initial_weights:Number|Iterable=0,
      initial_bias:Number=0,
      debug:bool=True
    ):
    self.debug = debug

    #> input size and learning rate

    # Input Size
    self.input_size = input_size
    self.input = np.repeat(0, self.input_size)
    
    # Learning Rate
    self.learning_rate = learning_rate

    #> activation, loss_function and optimizer

    params = {
      'activation':    activation,
      'loss_function': loss_function,
      'optimizer':     optimizer,
    }

    # iterate all
    for target in ['activation', 'loss_function', 'optimizer']:

      # get param
      param = params[target]
      
      # if object, set directly
      if type(param) == object: setattr(self, target, param)

      # if present in builtin, set
      elif type(param) == str and param in self.builtin[target].keys():
        setattr(self, target, self.builtin[target][param]())

      # raise error
      # if not in {object, str} or string and not in builtin
      else: raise self.errors[target](param)


    #> Initial Weights & Bias

    # initial weights
    if isinstance(initial_weights, Number):                                            self.weights = np.repeat(initial_weights, input_size).astype(float)
    elif isinstance(initial_weights, Iterable) and len(initial_weights) == input_size: self.weights = np.array(initial_weights).astype(float)
    else: raise InvalidInitialWeights(f'Value must be either a number or an Iterable of size {input_size}')

    # initialize gradient to zero
    self.weight_gradient = np.repeat(0.0, self.input_size)

    # initial bias
    self.bias = initial_bias

    if self.debug:
      print('initial')
      print('=======')
      print(f'weights={self.weights},\nbias={self.bias},\nlearning_rate={self.learning_rate}\n')


  
  #
  # Forward
  #

  def forward(self, input):
    self.input = np.array(input)

    # if 1d, predict
    if len(self.input.shape) and self.input.shape[0] == self.input_size:
      # weighted sum
      weighted_sum = np.dot(self.input, self.weights) + self.bias

      # activation
      self.output = self.activation.apply(weighted_sum)

      if self.debug: print(f'[DEBUG] input={input}, weighted_sum={weighted_sum}, predicted={self.output}')

      # return
      return self.output
    
    # elif 2d, predict multiple
    return self.forward_multiple(self.input)



  #
  # Forward Multiple samples
  #

  def forward_multiple(self, inputs):
    inputs = np.array(inputs)

    if len(inputs.shape) < 2 or inputs.shape[1] != self.input_size:
      raise UnexpectedInputsShape(f'Inputs shape must be (n, {self.input_size}). found {inputs.shape}')

    results = []
    for inp in inputs:
      results.append(self.forward(inp))

    return np.array(results)



  #
  # Backward Pass (loss & optimize)
  #

  def backward(self, error=None):
    self.error = error or self.error

    # gradient
    self.delta = self.error * self.activation.derivative(self.output)

    # gradients for weights and bias
    self.weight_gradient = self.delta * self.input
    self.bias_gradient = self.delta

    if self.debug: print(f'[DEBUG] delta={self.delta}, w_grad={self.weight_gradient}, b_grad={self.bias_gradient}')

    # return loss for previous neurons
    return np.dot(self.delta, self.weights)


  def update(self, learning_rate=None):
    lr = learning_rate or self.learning_rate

    if self.debug: print(f'[DEBUG] lr={lr}')

    if self.debug: print(f'[DEBUG] w_before={self.weights}, b_before={self.bias}')

    self.weights -= self.weight_gradient * lr
    self.bias    -= self.bias_gradient * lr

    if self.debug: print(f'[DEBUG] w_after={self.weights}, b_after={self.bias}')



  #
  # Train
  #

  def train(self, inputs:Iterable, outputs:Iterable, epochs:int=100, verbose:int=1):
    inputs = np.array(inputs)
    outputs = np.array(outputs)

    if len(inputs.shape) < 2 or inputs.shape[1] != self.input_size:    raise UnexpectedInputsShape (f'Inputs shape must be (n, {self.input_size}). found {inputs.shape}')
    if len(outputs.shape) != 1 or outputs.shape[0] != inputs.shape[0]: raise UnexpectedOutputsShape(f'Outputs size ({outputs.shape[0]}) differs from no of samples in input ({inputs.shape[0]})')
    
    for e in range(epochs):
      
      predicted = []
      
      for i in range(outputs.shape[0]):

        # Forward Pass
        self.forward(inputs[i])
        predicted.append(self.output)

        # error calculation
        self.error = self.output - outputs[i]

        if self.debug: print(f'[DEBUG] epoch={e+1}, sample={i+1}, actual={outputs[i]}, predicted={self.output}, error={self.error}')

        # Backward Pass
        self.backward()

        # Weights Updation
        self.optimizer.update(self)

        if verbose > 1: print(f'epoch={e+1}, sample={i+1}, actual={outputs[i]}, predicted={self.output}, error={self.error}')
      

      # Loss Calculation
      self.loss = self.loss_function.calculate(outputs, predicted)

      if verbose: print(f'epoch={e+1}, loss={self.loss}')



  #
  # Test
  #

  def test(self, inputs:Iterable, outputs:Iterable, verbose:int=0):
    inputs = np.array(inputs)
    outputs = np.array(outputs)

    if len(inputs.shape) < 2 or inputs.shape[1] != self.input_size:    raise UnexpectedInputsShape (f'Inputs shape must be (n, {self.input_size}). found {inputs.shape}')
    if len(outputs.shape) != 1 or outputs.shape[0] != inputs.shape[0]: raise UnexpectedOutputsShape(f'Outputs size ({outputs.shape[0]}) differs from no of samples in input ({inputs.shape[0]})')
    

    accurate = 0
    wrong    = 0
    for i in range(inputs.shape[0]):

      # Forward Pass
      predicted = self.forward(inputs[i])

      if predicted == outputs[i]: accurate+=1
      else:                       wrong   +=1

      if verbose > 1: print(f'sample={i+1}, predicted={predicted}, actual={outputs[i]}, error={outputs[i] - predicted}')

    return {
      'total':    accurate + wrong,
      'accurate': accurate,
      'wrong':    wrong,
    }


## Usage

In [31]:
# load dataset
from dataset import X, Y
X.shape

(10, 3)

#### Minimal

In [32]:
neuron = Neuron(3)

initial
weights=[0. 0. 0.],
bias=0,
learning_rate=0.1



In [33]:
neuron.train(X, Y)

[DEBUG] input=[10 15  1], weighted_sum=0.0, predicted=0.5
[DEBUG] epoch=1, sample=1, actual=1, predicted=0.5, error=-0.5
[DEBUG] delta=-0.11750185610079725, w_grad=[-1.17501856 -1.76252784 -0.11750186], b_grad=-0.11750185610079725
[DEBUG] lr=0.1
[DEBUG] w_before=[0. 0. 0.], b_before=0
[DEBUG] w_after=[0.11750186 0.17625278 0.01175019], b_after=0.011750185610079726
[DEBUG] input=[ 5 10  0], weighted_sum=2.3617873076260247, predicted=0.9138665965300853
[DEBUG] epoch=1, sample=2, actual=0, predicted=0.9138665965300853, error=0.9138665965300853
[DEBUG] delta=0.18669702494825818, w_grad=[0.93348512 1.86697025 0.        ], b_grad=0.18669702494825818
[DEBUG] lr=0.1
[DEBUG] w_before=[0.11750186 0.17625278 0.01175019], b_before=0.011750185610079726
[DEBUG] w_after=[ 0.02415334 -0.01044424  0.01175019], b_after=-0.006919516884746093
[DEBUG] input=[ 8 12  1], weighted_sum=0.07272652817393085, predicted=0.5181736225014092
[DEBUG] epoch=1, sample=3, actual=1, predicted=0.5181736225014092, error=-0.

In [34]:
neuron.test(X, Y)

[DEBUG] input=[10 15  1], weighted_sum=2.530488297743119, predicted=0.926251715703235
[DEBUG] input=[ 5 10  0], weighted_sum=-2.3710696767793484, predicted=0.08540554837734203
[DEBUG] input=[ 8 12  1], weighted_sum=2.0737616125098848, predicted=0.8883266678733718
[DEBUG] input=[3 5 0], weighted_sum=-1.9038927800725776, predicted=0.12966852255012332
[DEBUG] input=[ 7 14  1], weighted_sum=0.22856700149825748, predicted=0.5568942724172712
[DEBUG] input=[4 8 0], weighted_sum=-2.368457123910965, predicted=0.08560983980780305
[DEBUG] input=[ 9 13  1], weighted_sum=2.533100850611503, predicted=0.9264299792011766
[DEBUG] input=[2 4 0], weighted_sum=-2.3632320181741964, predicted=0.08601975186179958
[DEBUG] input=[6 9 0], weighted_sum=-0.9878268567377253, predicted=0.2713415277633772
[DEBUG] input=[11 16  1], weighted_sum=2.9898275358447366, predicted=0.952112447374728


{'total': 10, 'accurate': 0, 'wrong': 10}